In [1]:
import numpy as np
from scipy.stats import rayleigh
from scipy.special import digamma,erf
import scipy.io as sio
import pandas as pd

In [2]:
def parseMat(data):
    dataSeqs= IPTVData['Seqs']  
    mdtype = dataSeqs.dtype  
    seqs = {n: dataSeqs[n].flatten() for n in mdtype.names}
    dataStats = IPTVData['Stats']  
    mdtype = dataSeqs.dtype  
    stats = {n: dataSeqs[n].flatten() for n in mdtype.names}
    return seqs,stats

In [3]:
class Model:
    def __init__(self,seqs,K):
            self.N = len(seqs['Time'])
            D= np.zeros(self.N)
            sigma = np.zeros(self.N)
            Tmax = np.zeros(self.N)
            for i in range(self.N):
                D[i] = np.max(seqs['Mark'][i])
                sigma[i] = (4*(np.std(seqs['Time'][i])**5) / (3* len(seqs['Time'][i])))**0.2
                Tmax[i] = seqs['Time'][i][-1][0]
            self.D = np.int(np.max(D))
            Tmax = np.mean(Tmax)
            self.w = np.mean(sigma)
            self.landmark = self.w* range(np.int(np.ceil(Tmax/self.w)))
            self.alpha = np.ones(K)
            M = len(self.landmark)
            self.beta = np.ones((self.D,M,K,self.D)) / (M*(self.D**2))
            self.b = np.ones((self.D,K))/self.D
            label = np.round(K * np.random.rand(self.N))
            self.r = np.zeros((self.N,K))
            for k in range(K):
                self.r[label==k,k] = 1
            self.kernel = "gauss"
class Alg:
    def __init__ (self, outer = 8,rho = 0.1,inner = 5,thres =  1e-5,Tmax = []):
        self.outer = outer
        self.rho = rho
        self.inner = inner
        self.thres = thres
        self.Tmax = Tmax

In [4]:
linkedInData = sio.loadmat('Data/LinkedinData.mat')
IPTVData = sio.loadmat('Data/IPTVData.mat')
IPTVSeqs,IPTVStats = parseMat(IPTVData)
linkedInSeqs,linedInStats = parseMat(linkedInData)

In [15]:
def kernel_int(dt,model):
    distance = np.tile(dt.flatten(),(len(model.landmark),1))
    landmark = np.tile(model.landmark,(len(dt.flatten()),1)).T
    distance -= landmark
    G = 0
    if model.kernel == 'gauss':
        G = 0.5 *(erf(distance/(np.sqrt(2))*model.w)
                  + erf(landmark/(np.sqrt(2))*model.w))
    elif model.kernel == 'exp':
        G = 1 - np.exp(-model.w * (distance-landmark));
        G[G<0] = 0
    return G

In [12]:
def kernel(dt,model):
    distance = np.tile(dt.flatten(),(len(model.landmark),1))
    landmark = np.tile(model.landmark,(len(dt.flatten()),1)).T
    distance -= landmark
    g = 0
    if kernel == 'gauss':
        g = np.exp(-(distance**2)/(2*(model.w**2))/(np.sqrt(2*np.pi)*model.w))
    elif kernel == 'exp':
        g = model.w * np.exp(-model.w * distance)
        g[g>1] = 0
    return g

In [7]:
def E_log_pi(alpha):
    return digamma(alpha)  - digamma(np.sum(alpha))

In [19]:
K = 3
seqs = IPTVSeqs
model = Model(seqs,K)
alg = Alg()
Nk = np.sum(model.r,0)
alpha = model.alpha + Nk
LL = E_log_pi(alpha)
EX = np.zeros((model.N,K))
for c in range(model.N):
    time = seqs['Time'][c]
    event = seqs['Mark'][c]
    Tstart = seqs['Start'][c]
    
    if not alg.Tmax:
        Tstop = seqs['Stop'][c]
    else:
        Tstop = alg.Tmax
        indt = time < alg.Tmax
        time = time[indt]
        event = event[indt]
    N = len(time)
    G = kernel_int(Tstop-time,model)
    for i in range(N):
        ui = event[i]
        ti = time[i]
        break
    break

In [9]:
def Expectation_DMHP(Seqs,model,alg):
    

SyntaxError: unexpected EOF while parsing (<ipython-input-9-46420e66f3b4>, line 2)